In [2]:
from dotenv import load_dotenv
import os
import pygsheets
import requests
import pandas as pd

load_dotenv()

api_key = os.environ.get('riot_api_key')

In [3]:
api_key = os.environ.get('riot_api_key')
api_key

'RGAPI-aa50754f-76ed-46f2-bfa2-962b52806210'

In [4]:
service_account = pygsheets.authorize(
    service_account_file='JSONS\spreadsheet-automator-419713-f0c339d68def.json')

sheet = service_account.open_by_url(
    'https://docs.google.com/spreadsheets/d/1GoBz-6avCyXPqJeO6qz_hxrhPPW_tNPKcIp_G6dKAQ4/edit?usp=sharing')

In [5]:
from urllib import response


def get_puuid(summonerId=None, gameName=None, tagLine=None, region='br1'):
    if summonerId is not None:
        # Pega o puuid pelo summonerId ou pela gameTag+tagLine

        root_url = f'https://{region}.api.riotgames.com'
        endpoint = '/lol/summoner/v4/summoners/'
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)
        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        response = requests.get(root_url+endpoint+'?api_key='+api_key)
        return response.json()['puuid']


def get_idtag_from_puuid(puuid=None, region='br1'):
    # Pega o riot_id e o id_tag pelo puuid

    root_url = f'https://{region}.api.riotgames.com'
    endpoint = 'riot/account/v1/accounts/by-puuid/'
    response = requests.get(root_url+endpoint+puuid+'?api_key='+api_key)

    id = {
        'gameName': response.json()['gameName'],
        'tagLine': response.json()['tagLine']
    }

    return id


def get_ladder(top=None, region='br1'):
    #Pega a ladder top 300 ou mais
    root = f'https://{region}.api.riotgames.com'
    chall = '/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = '/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = '/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(root+chall+'?api_key='+api_key)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values(
        'leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame()
    master_df = pd.DataFrame()

    if top > 300:
        gm_response = requests.get(root+gm+'?api_key='+api_key)
        gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values(
        'leaguePoints', ascending=False).reset_index(drop=True)
    if top > 1000:
        master_response = requests.get(root+master+'?api_key='+api_key)
        master_df = pd.DataFrame(master_response.json()['entries']).sort_values(
        'leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, master_df]).reset_index(drop=True)[:top]
    ladder = ladder.drop(columns='rank').reset_index(
        drop=False).rename(columns={'index': 'rank'})
    ladder['rank'] += 1

    return ladder

def get_match_history(region=None, puuid=None, count=20):
    #Pega o historico de 20 partidas 
    root = f'https://{region}.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    count = f'?start=0&count={count}&'
    response = requests.get(root+endpoint+count+'&api_key='+api_key)

    return response.json()

def get_match(region=None, matchId=None):
    #Pega informações sobre uma partida especifica
    root = f'https://{region}.api.riotgames.com'
    endpoint = f'/lol/match/v5/matches/{matchId}'
    response = requests.get(root+endpoint+'?api_key='+api_key)

    return response.json()

In [6]:
get_match_history(region='americas', puuid='nLcOi3gpM4WUybwDjCXHKeW1qDLehxlGOxgRyxpWi9f7HxfAAFEwqlCXFsM-y0J5MqyqsY8y79xg2g')

['BR1_2924848006',
 'BR1_2924825088',
 'BR1_2924797780',
 'BR1_2924764078',
 'BR1_2924729032',
 'BR1_2924694209',
 'BR1_2922425243',
 'BR1_2918466941',
 'BR1_2918259478',
 'BR1_2918216826',
 'BR1_2918181366',
 'BR1_2918156928',
 'BR1_2918122030',
 'BR1_2918082594',
 'BR1_2918071191',
 'BR1_2918041908',
 'BR1_2918029938',
 'BR1_2917801140',
 'BR1_2917762246',
 'BR1_2917740062']

In [7]:
game = get_match(region='americas', matchId='BR1_2918466941')

In [8]:
get_puuid(summonerId='g250Cv7oaxE9gb-eysxOwyasvT7kJje3uMDDOJsiVnhqabg')

'nLcOi3gpM4WUybwDjCXHKeW1qDLehxlGOxgRyxpWi9f7HxfAAFEwqlCXFsM-y0J5MqyqsY8y79xg2g'

----------------------------------------
Informações que quero retirar da partida:
Match > MetadataDto:
- matchId
- participants	

Match > InfoDto:
- endOfGameResult
- gameCreation
- gameDuration
- gameEndTimestamp
- gameVersion as PATCH

Match > InfoDto > ParticipantDto:
- assists
- champLevel
- championName
- damageDealtToBuildings
- damageDealtToObjectives
- damageDealtToTurrets
- deaths
- detectorWardsPlaced
- firstTowerKill	
- firstBloodKill
- goldEarned
- inhibitorKills
- gameEndedInSurrender
- gameEndedInEarlySurrender
- item0	
- item1	
- item2	
- item3	
- item4	
- item5	
- item6
- neutralMinionsKilled
- objectivesStolen
- participantId
- puuid
- riotIdName
- riotIdTagline
- summoner1Id
- summoner2Id
- summonerId
- summonerName
- totalDamageDealtToChampions
- totalDamageShieldedOnTeammates
- totalDamageTaken
- totalHealsOnTeammates
- turretKills
- turretsLost
- visionScore
- wardsKilled
- wardsPlaced
- win

Match > InfoDto > ParticipantDto > PerksDto > PerksStatsDto:
- defense
- flex
- offense

Match > InfoDto > ParticipantDto > PerksDto > PerksStyleDto:
- style

Match > InfoDto > ParticipantDto > PerksDto > PerksStyleDto > PerkStyleSelectionDto: 
- perk	
- var1	
- var2	
- var3

Match > TeamDto > BanDto:
- championId

Match > TeamDto > ObjectivesDto:
- baron		
- champion		
- dragon		
- inhibitor		
- riftHerald		
- tower	

In [33]:
metadata = game['metadata']
info = game['info']
players = info['participants']
endGame = info['endOfGameResult']
gameCreation = info['gameCreation']
gameDuration = info['gameDuration']
gameEndTimestamp = info['gameEndTimestamp']
patch = info['gameVersion']

matchid = metadata['matchId']

teams = info['teams']


for player in players:
  assists = player['assists']
  champLevel = player['champLevel']
  champName = player['championName']
  damageToBuilding = player['damageDealtToBuildings']
  damageToObjectives = player['damageDealtToObjectives']
  damageToTurrets = player['damageDealtToTurrets']
  deaths = player['deaths']
  wardsPlaced = player['detectorWardsPlaced']
  firstTower = player['firstTowerKill']
  firstBlood = player['firstBloodKill']
  goldEarned = player['goldEarned']
  inhibitors = player['inhibitorKills']
  surrender = player['gameEndedInSurrender']
  earlySurrender = player['gameEndedInEarlySurrender']
  item0 = player['item0']
  item1 = player['item1']
  item2 = player['item2']
  item3 = player['item3']
  item4 = player['item4']
  item5 = player['item5']
  item6 = player['item6']
  minionsKilled = player['neutralMinionsKilled']
  objectiveStolen = player['objectivesStolen']
  participantId = player['participantId']
  riotIdName = player['riotIdName']
  #idTagLine = player['riotIdTagLine']
  summoner1Id = player['summoner1Id']
  summoner2Id = player['summoner2Id']
  summonerId = player['summonerId']
  damageToChamps = player['totalDamageDealtToChampions']
  damgeOnShield = player['totalDamageShieldedOnTeammates']
  damageTaken = player['totalDamageTaken']
  heal = player['totalHealsOnTeammates']
  turretKills = player['turretKills']
  turretLosts = player['turretsLost']
  visionScore = player['visionScore']
  wardsKilled = player['wardsKilled']
  wardsPlaced = player['wardsPlaced']
  win = player['win']
  

  perks = player['perks']
  stats = perks['statPerks']
  defense = stats['defense']
  offense = stats['offense']
  flex = stats['flex']

  styles = perks['styles']
  

  primary_style = styles[0]
  secondary_style = styles[1]

for team in teams:
  #bans = team['bans']

  if team['teamId'] == player['teamId']:
    objectives = team['objectives']
    baron = objectives['baron']
    dragon = objectives['dragon']
    champion = objectives['champion']
    inhibitor = objectives['inhibitor']
    riftHerald = objectives['riftHerald']
    tower = objectives['tower']
    grubs = objectives['horde']


KeyError: 'riotIdName'

In [38]:
print(tower)
print(champion)
print(riftHerald)
print(grubs)
print(baron)
print(dragon)

{'first': False, 'kills': 9}
{'first': True, 'kills': 30}
{'first': False, 'kills': 0}
{'first': True, 'kills': 2}
{'first': True, 'kills': 2}
{'first': True, 'kills': 2}


In [39]:
death = game['info']['participants']['']
print(death)

TypeError: list indices must be integers or slices, not str